In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.DESCR)

In [ ]:
X = boston.data
Y = boston.target
print(X.shape) #design matrix
print(Y.shape)

In [ ]:
import numpy as np
import torch
np.random.seed(123) #in produzione si deve fissare un seed
torch.random.manual_seed(123);
idx = np.random.permutation(len(X))

In [ ]:
X = X[idx]
Y = Y[idx]

In [ ]:
X_training = torch.Tensor(X[50:])
Y_training = torch.Tensor(Y[50:])
X_testing = torch.Tensor(X[:50])
Y_testing = torch.Tensor(Y[:50])

In [ ]:
theta = torch.Tensor(13) #creiamo un tensore di 13 unità
theta_0 = torch.Tensor(1) #tensore di una unità (bias)
#impostiamo required_grad
theta.requires_grad_(True)
theta_0.requires_grad_(True)
theta.data.normal_(0,0.1) #inizializziamo il tensore con numeri casuali tratti da una
#distribuzione normale di media 0 e varianza 0.1
theta_0.data.normal_(0,0.1) 
#media 0 perchè non vogliamo dare un bias ai parametri
#varianza piccola attorno allo 0 riduce il bias che diamo ai parametri
#le diverse componenti vivono nello stesso spazio

print(theta)
print(theta_0)

In [ ]:
def linear_regression(input, theta, theta_0):
    return input.mul(theta).sum(1)+theta_0

In [ ]:
x=X_training[0]

In [ ]:
print(x.mul(theta).sum(1))

In [ ]:
print(X_training.mul(theta).sum(1)[0]+theta_0)

In [ ]:
y = linear_regression(X_training,theta,theta_0)
print(y[:10]) #stampiamo i primi 10 valori predetti

In [ ]:
def loss(input, target):
    return ((input-target)**2).mean()

In [ ]:
print(loss(y,Y_training)) #l'unità di misura è dollari al quadrato

In [ ]:
means = X_training.mean(0)
stds = X_training.std(0)
print(means, stds)

La normalizzazione la effettuiamo utilizzando la media e la deviazione standard del set di training. Tre motivi:
- Se ricalcolo le statistiche dal test set potrebbero trovarsi in un altro spazio rispetto a quelli di training 
- Ricalcolare le statistiche significa che dobbiamo ritrainare i parametri
- I dati di test si presentano ad uno ad uno e quindi le statistiche non possono essere calcolate su di essi

In [ ]:
lr = 0.1
epochs = 10 #iterazioni
# Passo 1: normalizzazione dei dati
means = X_training.mean(0)
stds = X_training.std(0)
X_training_norm = (X_training-means)/stds
X_testing_norm = (X_testing-means)/stds
#Passo 2: inizializziamo i pesi come visto in precedenza
theta = torch.Tensor(13);
theta_0 = torch.Tensor(1);
theta.normal_(0,0.01)
theta_0.normal_(0,0.01)
theta.requires_grad_(True)
theta_0.requires_grad_(True)
for e in range(epochs):
    #Passo 3: calcoliamo le predizioni
    y = linear_regression(X_training_norm,theta,theta_0)

    #Passo 4: calcoliamo il valore della loss
    l = loss(y, Y_training)
    #Passo 5: calcoliamo il gradiente della loss rispetto a tutti i parametri
    l.backward()
    #stampiamo il valore della loss
    print("Epoch: {}, loss: {:0.2f}".format(e,l.item()))

    #Passo 6: Aggiorniamo i pesi
    theta.data.sub_(lr*theta.grad.data)
    theta_0.data.sub_(lr*theta_0.grad.data)

    #azzeriamo i gradienti per evitare di accumularli
    theta.grad.data.zero_()
    theta_0.grad.data.zero_()

In [ ]:
lr = 0.1
epochs = 50
# Passo 1: normalizzazione dei dati
means = X_training.mean(0)
stds = X_training.std(0)
X_training_norm = (X_training-means)/stds
X_testing_norm = (X_testing-means)/stds
#Passo 2: inizializziamo i pesi come visto in precedenza
theta = torch.Tensor(13);
theta_0 = torch.Tensor(1);
theta.normal_(0,0.01)
theta_0.normal_(0,0.01)
theta.requires_grad_(True)
theta_0.requires_grad_(True)
losses = []
for e in range(epochs):
    #Passo 3: calcoliamo le predizioni
    y = linear_regression(X_training_norm,theta,theta_0)

    #Passo 4: calcoliamo il valore della loss
    l = loss(y, Y_training)
    #Passo 5: calcoliamo il gradiente della loss rispetto a tutti i parametri
    l.backward()
    #conserviamo il valore della loss
    losses.append(l.item())
    #Passo 6: Aggiorniamo i pesi
    theta.data.sub_(lr*theta.grad.data)
    theta_0.data.sub_(lr*theta_0.grad.data)

    #azzeriamo i gradienti per evitare di accumularli
    theta.grad.data.zero_()
    theta_0.grad.data.zero_()

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(12,8))
plt.plot(losses)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()

In [ ]:
#iniziamo calcolando le predizioni del modello dati i pesi allenati
yt = linear_regression(X_testing_norm,theta,theta_0)
#calcoliamo il valore della loss
print(loss(yt, Y_testing))

In [ ]:
lr = 0.1
epochs = 50
# Passo 1: normalizzazione dei dati
means = X_training.mean(0)
stds = X_training.std(0)
X_training_norm = (X_training-means)/stds
X_testing_norm = (X_testing-means)/stds
#Passo 2: inizializziamo i pesi come visto in precedenza
theta = torch.Tensor(13);
theta_0 = torch.Tensor(1);
theta.normal_(0,0.01)
theta_0.normal_(0,0.01)
theta.requires_grad_(True)
theta_0.requires_grad_(True)
losses_train = []
losses_test = []
for e in range(epochs):
    #Passo 3: calcoliamo le predizioni
    y = linear_regression(X_training_norm,theta,theta_0)

    #Passo 4: calcoliamo il valore della loss
    l = loss(y, Y_training)
    #conserviamo il valore della loss di training
    losses_train.append(l.item())
    #Passo 5: calcoliamo il gradiente della loss rispetto a tutti i parametri
    l.backward()
    #Passo 6: Aggiorniamo i pesi
    theta.data.sub_(lr*theta.grad.data)
    theta_0.data.sub_(lr*theta_0.grad.data)

    #azzeriamo i gradienti per evitare di accumularli
    theta.grad.data.zero_()
    theta_0.grad.data.zero_()

    #calcoliamo la loss sul test set
    #dato che non dobbiamo calcolare i gradienti
    #li disabilitiamo per risparmiare memoria
    #questa notazione è di pytorch per disabilitare i gradienti
    with torch.set_grad_enabled(False):
        y_test = linear_regression(X_testing_norm, theta, theta_0)
        loss_test = loss(y_test, Y_testing)
        losses_test.append(loss_test.item())

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(12,8))
plt.plot(losses_train)
plt.plot(losses_test)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.legend(['Training','Testing'])
plt.show()

Esercizio. y = Ax + b. Dove stanno le feature e dove le variabili?

In [ ]:
from torch import nn
linear = nn.Linear(20,10)

In [ ]:
par = list(linear.parameters())
print(par[0].shape) # Matrice A [m x n]
print(par[1].shape) # Vettore b [m]

In [ ]:
#costruiamo una matrice di 150 elementi a 20 dimensioni
sample_input = torch.rand((150,20))
#otteniamo una matrice di 150 elementi a 10 dimensioni
sample_output = linear(sample_input)
# abbiamo svolto l'operazione y = Ax + b
print(sample_output.shape)

In [ ]:
linreg = nn.Linear(13,1)
z = linreg(X_training_norm)
print(z.shape)

optimizer.step() #ottimizzazione <br>
optimizer.zero_grad() #sostituisce il data.zero_grad

<h1>Esercizio 1</h1>
<p>Si provi ad allenare il regressore sul dataset Boston senza effettuare la normalizzazione dei dati. Se
necessario, si modifichino il learning rate e il numero di epoche per portare il modello a convergenza.
Si notano differenze durante il training? Il modello allenato è migliore o peggiore? Si monitorino le
curve di training e test mediante tensorboard.</p>

In [ ]:
from sklearn.datasets import load_boston
import numpy as np
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

class LinearRegressor(nn.Module):
    def __init__(self, in_size, out_size):
        """Costruisce un regressore logistico.
        Input:
        in_size: numero di feature in input (es. 13)
        out_size: numero di elementi in output (es. 1)"""
        super(LinearRegressor, self).__init__() #richiamo il costruttore della superclasse
        #questo passo è necessario per abilitare alcuni meccanismi automatici dei moduli di PyTorch
        self.linear = nn.Linear(in_size,out_size)
    def forward(self,x):
        """Definisce come processare l'input x"""
        result = self.linear(x)
        return result
    
def reset_elements():
    boston = load_boston()
    X = boston.data
    Y = boston.target
    np.random.seed(123)
    torch.random.manual_seed(123);
    idx = np.random.permutation(len(X))
    X = X[idx]
    Y = Y[idx]
    X_training = torch.Tensor(X[50:])
    Y_training = torch.Tensor(Y[50:])
    X_testing = torch.Tensor(X[:50])
    Y_testing = torch.Tensor(Y[:50])
    return X_training, Y_training, X_testing, Y_testing

def esercizio1(lr, epochs, type, now):
    #facciamo training per 5000 epoche in modo da verificare che
    #tensorboard viene aggiornato in tempo reale
    writer = SummaryWriter('logs/lab_1/esercizio1/{}/{}'.format(type,now))
    X_training, Y_training, X_testing, Y_testing = reset_elements()
    if type is "norm":
        #normalizzazione dei dati
        means = X_training.mean(0)
        stds = X_training.std(0)
    else:
        means = 0
        stds = 1
    X_training_norm = (X_training-means)/stds
    X_testing_norm = (X_testing-means)/stds
    reg = LinearRegressor(13,1)
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(reg.parameters(),lr=lr)
    for e in range(epochs):
        reg.train()
        output = reg(X_training_norm)
        l = criterion(output.view(-1),Y_training)
        writer.add_scalar('loss/train', l.item(), global_step=e)
        l.backward()
        optimizer.step()
        optimizer.zero_grad()
        reg.eval()
        with torch.set_grad_enabled(False):
            y_test = reg(X_testing_norm)
            l = criterion(y_test.view(-1),Y_testing)
            writer.add_scalar('loss/test', l.item(), global_step=e)

In [ ]:
import datetime
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S%z")
epochs = 1000
lr = 10**-1
esercizio1(lr, epochs, "norm", now)
epochs = 50000
lr = 3*(10**-6)
esercizio1(lr, epochs, "not_norm", now)

Senza normalizzazione dopo 50000 epochs ancora non si è andati in convergenza (che è una quantità almeno 250 volte più grande del numero di epoche richieste se usiamo la normalizzazione).

<h1>Esercizio 2</h1>
<p>
    Abbiamo visto come allenare un modello di regressione lineare in maniera imperativa. Per
semplificare l'utilizzo del modello, si cotruisca una classe LinearRegressor con i seguenti
metodi:
<ul>
    <li>Costruttore: prende in input il numero di osservazioni . Il costruttore inizializza i pesi del modello
(theta e theta_0);</li>
    <li>Metodo fit : prende in input i dati
e le etichette
per effettuare il training. Il metodo prende in
input anche i parametri lr e epochs che indicano il learning rate e il numero di epoche. I valori di default per questi due
parametri sono rispettivamente
e
. Il metodo fit calcola medie e deviazioni standard di
e conserva tali valori
per usi futuri, normalizza i dati
ed effettua l'allenamento del modello. Ad ogni epoca, viene stampato il valore della loss;</li>
    <li>Metodo predict : prende in input i dati . Il metodo normalizza i dati
utilizzando le medie e le deviazioni standard
precedentemente salvate, poi predice e restituisce le etichette predette dal modello sui dati ;</li>
    <li>Metodo score : prende in input i dati
e le etichette . Il metodo utilizza predict per predire le etichette a partire dai
dati
, poi calcola e restituisce il valore della loss calcolata utilizzando le etichette predette e le etichette fornite
;</li>
</ul>
E' possibile inserire altri metodi privati (devono iniziare per _ , ad esempio \_loss ) per rendere la computazione modulare.
Utilizzare l'oggetto per:
<ul>
    <li>
Allenare il modello di regressione lineare sui dati di training;</li>
    <li>
Calcolare la loss di training mediante il metodo score ;</li>
    <li>
Predire le etichette di test mediante il metodo predict ;</li>
    <li>
Calcolare la loss di test mediante il metodo score .</li>
</ul>
</p>

In [ ]:
from sklearn.datasets import load_boston
import numpy as np
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

def reset_elements():
    boston = load_boston()
    X = boston.data
    Y = boston.target
    np.random.seed(123)
    torch.random.manual_seed(123);
    idx = np.random.permutation(len(X))
    X = X[idx]
    Y = Y[idx]
    X_training = torch.Tensor(X[50:])
    Y_training = torch.Tensor(Y[50:])
    X_testing = torch.Tensor(X[:50])
    Y_testing = torch.Tensor(Y[:50])
    return X_training, Y_training, X_testing, Y_testing

class LinearRegressor(nn.Module):
    def __init__(self, in_size, out_size):
        """Costruisce un regressore lineare.
        Input:
        in_size: numero di feature in input (es. 13)
        out_size: numero di elementi in output (es. 1)"""
        super(LinearRegressor, self).__init__() #richiamo il costruttore della superclasse
        #questo passo è necessario per abilitare alcuni meccanismi automatici dei moduli di PyTorch
        self.linear = nn.Linear(in_size,out_size)
        self.means = 0
        self.stds = 1
        self.criterion = nn.MSELoss()
        self.writer = SummaryWriter('logs/esercizio2/')
        
    def forward(self,x):
        """Definisce come processare l'input x"""
        result = self.linear(x)
        return result
    
    def _norm(self, X):
        return (X-self.means)/self.stds
    
    def predict(self, X):
        return self.forward(self._norm(X)).view(-1)
    
    def score(self, X, Y):
        return self.criterion(self.predict(X),Y).item()
        
    
    def fit(self, X, Y, lr=0.01, epochs=100):
        self.means = X.mean(0)
        self.stds = X.std(0)
        X_norm = self._norm(X)
        optimizer = torch.optim.SGD(self.parameters(),lr=lr)
        for e in range(epochs):
            self.train()
            output = self.forward(X_norm)
            l = self.criterion(output.view(-1),Y)
            print("loss/fit: {}".format(l.item()))
            self.writer.add_scalar('loss/fit', l.item(), global_step=e)
            l.backward()
            optimizer.step()
            optimizer.zero_grad()

In [ ]:
reg = LinearRegressor(13,1)
X_training, Y_training, X_testing, Y_testing = reset_elements()

In [ ]:
X_training, Y_training, X_testing, Y_testing = reset_elements()
reg.fit(X_training, Y_training, 0.1, 5000)

In [ ]:
print(reg.score(X_training, Y_training))

In [ ]:
predictions = reg.predict(X_testing)
print(predictions)
print(Y_testing)

In [ ]:
print(reg.score(X_testing, Y_testing))

<h1>Esercizio 3</h1>
<p>
    Si consideri il dataset disponibile al seguente URL:<br>
https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset<br>
Si costruisca un regressore lineare che predica i valori target a partire dagli altri attributi. Si provino
diversi learning rate e numeri di epoche per far convergere il modello. Cosa succede nel caso di
learning rate molto alti? E nel caso di learning rate molto bassi?
</p>

In [ ]:
from sklearn.datasets import load_diabetes
import numpy as np
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

def reset_elements():
    diabetes = load_diabetes()
    X = diabetes.data
    Y = diabetes.target
    np.random.seed(123)
    torch.random.manual_seed(123);
    idx = np.random.permutation(len(X))
    X = X[idx]
    Y = Y[idx]
    X_training = torch.Tensor(X[40:])
    Y_training = torch.Tensor(Y[40:])
    X_testing = torch.Tensor(X[:40])
    Y_testing = torch.Tensor(Y[:40])
    return X_training, Y_training, X_testing, Y_testing

class LinearRegressor(nn.Module):
    def __init__(self, in_size, out_size):
        """Costruisce un regressore lineare.
        Input:
        in_size: numero di feature in input (es. 10)
        out_size: numero di elementi in output (es. 1)"""
        super(LinearRegressor, self).__init__() #richiamo il costruttore della superclasse
        #questo passo è necessario per abilitare alcuni meccanismi automatici dei moduli di PyTorch
        self.linear = nn.Linear(in_size,out_size)
        self.means = 0
        self.stds = 1
        self.criterion = nn.MSELoss()
        self.writer = SummaryWriter('logs/esercizio3/')
        
    def forward(self,x):
        """Definisce come processare l'input x"""
        result = self.linear(x)
        return result
    
    def _norm(self, X):
        return (X-self.means)/self.stds
    
    def predict(self, X):
        return self.forward(self._norm(X)).view(-1)
    
    def score(self, X, Y):
        return self.criterion(self.predict(X),Y).item()
        
    
    def fit(self, X, Y, lr=0.01, epochs=100):
        self.means = X.mean(0)
        self.stds = X.std(0)
        X_norm = self._norm(X)
        optimizer = torch.optim.SGD(self.parameters(),lr=lr)
        for e in range(epochs):
            self.train()
            output = self.forward(X_norm)
            l = self.criterion(output.view(-1),Y)
            #print("loss/fit: {}".format(l.item()))
            self.writer.add_scalar('lr={}/training'.format(lr), l.item(), global_step=e)
            l.backward()
            optimizer.step()
            optimizer.zero_grad()

In [ ]:
for i in range(-10,11):
    reg = LinearRegressor(10,1)
    X_training, Y_training, X_testing, Y_testing = reset_elements()
    reg.fit(X_training, Y_training, 1*(10**i), 3000)

Lr bassissimi -> arriva molto lentamente alla convergenza
Lr altissimi >= 1 -> non converge mai, oscilla (o infinito)

<h1>Esercizio 4</h1>
<p>
    Si consideri il dataset visto nell'esercizio precedente. Si costruisca un regressore lineare per predire i
valori delle variabili S1 , S2 , S3 a partire dai valori delle altre variabili. Per costruire il regressore,
si specifichi
3
come numero di dimensioni in uscita. Si faccia attenzione al processo di
normalizzazione dei dati. In questo caso, i valori delle variabili S1 , S2 , S3 non deve essere
normalizzato. Qual è la loss finale sul test set?
</p>

In [22]:
from sklearn.datasets import load_diabetes
import numpy as np
import torch
import pandas as pd
from torch import nn
from torch.utils.tensorboard import SummaryWriter

def reset_elements():
    diabetes = load_diabetes()
    df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
    X = df[['age','sex','bmi','bp','s4','s5','s6']].copy().to_numpy()
    Y = df[['s1','s2','s3']].copy().to_numpy()
    np.random.seed(123)
    torch.random.manual_seed(123);
    idx = np.random.permutation(len(X))
    X = X[idx]
    Y = Y[idx]
    X_training = torch.Tensor(X[40:])
    Y_training = torch.Tensor(Y[40:])
    X_testing = torch.Tensor(X[:40])
    Y_testing = torch.Tensor(Y[:40])
    return X_training, Y_training, X_testing, Y_testing

class LinearRegressor(nn.Module):
    def __init__(self, in_size, out_size):
        """Costruisce un regressore lineare.
        Input:
        in_size: numero di feature in input (es. 10)
        out_size: numero di elementi in output (es. 1)"""
        super(LinearRegressor, self).__init__() #richiamo il costruttore della superclasse
        #questo passo è necessario per abilitare alcuni meccanismi automatici dei moduli di PyTorch
        self.linear = nn.Linear(in_size,out_size)
        self.means = 0
        self.stds = 1
        self.criterion = nn.MSELoss()
        self.writer = SummaryWriter('logs/esercizio4/')
        
    def forward(self,x):
        """Definisce come processare l'input x"""
        result = self.linear(x)
        return result
    
    def _norm(self, X):
        return (X-self.means)/self.stds
    
    def predict(self, X):
        return self.forward(self._norm(X)).view(-1)
    
    def score(self, X, Y):
        return self.criterion(self.predict(X),Y).item()
        
    
    def fit(self, X, Y, lr=0.01, epochs=100):
        self.means = X.mean(0)
        self.stds = X.std(0)
        X_norm = self._norm(X)
        optimizer = torch.optim.SGD(self.parameters(),lr=lr)
        for e in range(epochs):
            self.train()
            output = self.forward(X_norm)
            l = self.criterion(output.view(-1),Y.view(-1))
            #print("loss/fit: {}".format(l.item()))
            self.writer.add_scalar('lr={}/fit'.format(lr), l.item(), global_step=e)
            l.backward()
            optimizer.step()
            optimizer.zero_grad()

In [23]:
reg = LinearRegressor(7,3)
X_training, Y_training, X_testing, Y_testing = reset_elements()
reg.fit(X_training, Y_training, 1*(10**-1), 3000)